In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy

2023-12-01 23:35:45.677843: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 23:35:45.733795: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 23:35:45.733834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 23:35:45.735243: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 23:35:45.741498: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 23:35:45.742380: I tensorflow/core/platform/cpu_feature_guard.cc:1

### Preparing Dataset

In [2]:
train_dataset = keras.utils.image_dataset_from_directory(
    './seg_train', batch_size=64, image_size=(150, 150))

validation_dataset = keras.utils.image_dataset_from_directory(
    './seg_test', batch_size=64, image_size=(150, 150))

# prediction_dataset = keras.utils.image_dataset_from_directory(
#     './seg_pred', batch_size=64, image_size=(150, 150))

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


### Model Definition

In [3]:
model = Sequential([
    Convolution2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Convolution2D(filters=32, kernel_size=(3, 3), activation='relu'),
    Flatten(input_shape=(150, 150, 3)),
    # Dense(units=16, activation='relu'),
    # Dense(units=32, activation='relu'),
    Dense(units=6, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        9248      
                                                                 
 flatten (Flatten)           (None, 165888)            0         
                                                                 
 dense (Dense)               (None, 6)                 995334    
                                                                 
Total params: 1005478 (3.84 MB)
Trainable params: 1005478 (3.84 MB)
Non-trainable params: 0 (0.00 Byte)
__________________

In [4]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [5]:
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10,
    verbose=2)


Epoch 1/10
220/220 - 186s - loss: 71.8612 - accuracy: 0.3254 - val_loss: 1.5576 - val_accuracy: 0.4223 - 186s/epoch - 848ms/step
Epoch 2/10
220/220 - 180s - loss: 1.2087 - accuracy: 0.5336 - val_loss: 1.5989 - val_accuracy: 0.4483 - 180s/epoch - 820ms/step
Epoch 3/10
220/220 - 175s - loss: 0.7857 - accuracy: 0.7066 - val_loss: 1.8275 - val_accuracy: 0.4523 - 175s/epoch - 797ms/step
Epoch 4/10
220/220 - 197s - loss: 0.5410 - accuracy: 0.8099 - val_loss: 1.9504 - val_accuracy: 0.4337 - 197s/epoch - 893ms/step
Epoch 5/10


KeyboardInterrupt: 

## Prediction

In [ ]:
predictions = model.predict(prediction_dataset, verbose=2)

## Visualizing

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred = np.argmax(predictions, axis=1)
y_true = np.concatenate([y for x, y in prediction_dataset], axis=0)
cm = confusion_matrix(y_true, y_pred)